# DSCI 504

## Week 2: Building the OPC Database

### Student Name: SEIF KUNGULIO

Assignment Objectives:

1. Build a database schema
2. Build database tables
3. Identify and establish Primary and Foreign Keys
4. Identify and establish key table Constraints
5. Import data into PostgreSQL
6. Query a table

**Step 1: Create a new schema named 'dsci\_504'.**

In [1]:
--Enter your SQL code here
CREATE SCHEMA dsci_504

CREATE SCHEMA

Total execution time: 00:00:00.007

**STEP 2: Create the necessary tables identified in the Assignemt 2 Table Structure document located in Canvas. Use a new code cell for each table.**

In [2]:
--Enter your SQL code here. Use a single cell for each table/task. Do not enter all tables in one cell. It will be harder to troublsehoot errors.
-- 1) Lookup: STATES
CREATE TABLE dsci_504.states (
  state_id   INT   PRIMARY KEY --The numerical state id from 1 to 50
  ,state     TEXT  NOT NULL --The two-digit state digraph
);

CREATE TABLE

Total execution time: 00:00:00.024

In [3]:
-- 2) CUSTOMERS
CREATE TABLE dsci_504.customers (
  cus_id         SERIAL PRIMARY KEY --The numerical customer ID
  ,cus_num        TEXT --A unique customer number derived from the customer name and a generated number
  ,cus_last_name  TEXT --Customer Last Name
  ,cus_first_name TEXT --Customer First Name
  ,cus_add_num    TEXT --Numerical house number
  ,cus_address    TEXT --Line 2 address
  ,cus_city       TEXT --Customer City
  ,state_id       INT  NOT NULL REFERENCES dsci_504.states(state_id) --State ID
  ,cus_zip        TEXT --5-digit zip code
  ,cus_phone      TEXT --10-digit telephone
  ,cus_join_date  DATE --Customer join date
  ,cus_app_cd     TEXT --Customer appreication code (1-7)
  ,cus_app_num    TEXT --Customer loyalty program number
);

CREATE TABLE

Total execution time: 00:00:00.025

In [4]:
-- 3) WAREHOUSES
CREATE TABLE dsci_504.warehouses (
  warehouse_id   SERIAL PRIMARY KEY --Warehouse ID
  ,warehouse_name TEXT  NOT NULL --Full city name of warehouse
  ,state_id       INT   NOT NULL REFERENCES dsci_504.states(state_id) --Integer State ID
);

CREATE TABLE

Total execution time: 00:00:00.022

In [5]:
-- 4) SUPPLIERS
CREATE TABLE dsci_504.suppliers (
  sup_id   SERIAL PRIMARY KEY --Supplier ID number
  ,sup_name TEXT  NOT NULL --Supplier Text Name
  ,sup_ctry TEXT --Supplier Country(s)
);

CREATE TABLE

Total execution time: 00:00:00.035

In [6]:
-- 5) COMPONENTS
CREATE TABLE dsci_504.components (
  comp_id    SERIAL PRIMARY KEY --Component ID Number
  ,comp_name  TEXT  NOT NULL --Name of Component
  ,comp_cost  NUMERIC(10,2) --Cost of Component
  ,sup_id     INT   REFERENCES dsci_504.suppliers(sup_id) --Supplier Integer ID
  ,comp_cat   TEXT --Component category (bucketed)
);

CREATE TABLE

Total execution time: 00:00:00.013

In [7]:
-- 6) BUILDS
CREATE TABLE dsci_504.builds (
  build_id   SERIAL PRIMARY KEY --Build ID
  ,build_name TEXT  NOT NULL --Build name
);

CREATE TABLE

Total execution time: 00:00:00.054

In [8]:
-- 7) BUILD_COMPONENTS (many-to-many Builds ↔ Components)
CREATE TABLE dsci_504.build_components (
  build_id INT NOT NULL REFERENCES dsci_504.builds(build_id) --Integer Build ID
  ,comp_id INT NOT NULL REFERENCES dsci_504.components(comp_id) --Integer Component ID
  ,PRIMARY KEY (build_id, comp_id)
);

CREATE TABLE

Total execution time: 00:00:00.060

In [9]:
-- 8) PRODUCTS
CREATE TABLE dsci_504.products (
  prod_id           SERIAL PRIMARY KEY --Product ID
  ,prod_cat_name     TEXT --Product Category Name
  ,prod_manufacturer TEXT --Product Manufacturer (could be Supplier)
  ,prod_name         TEXT NOT NULL --Product Name
  ,prod_description  TEXT --Product Description
  ,prod_price        NUMERIC(10,2) --Product Price
  ,prod_class        TEXT --Product Classification
  ,country_origin    TEXT --Country of Origin
);

CREATE TABLE

Total execution time: 00:00:00.021

In [10]:
-- 9) PRODUCT_BUILDS (many-to-many Products ↔ Builds)
CREATE TABLE dsci_504.product_builds (
  prod_id  INT NOT NULL REFERENCES dsci_504.products(prod_id) --Integer Product ID
  ,build_id INT NOT NULL REFERENCES dsci_504.builds(build_id) --Integer Build ID
  ,PRIMARY KEY (prod_id, build_id)
);

CREATE TABLE

Total execution time: 00:00:00.042

In [11]:
-- 10) ORDERS (header)
CREATE TABLE dsci_504.orders (
  ord_id        SERIAL PRIMARY KEY --Order ID
  ,ord_date      DATE  NOT NULL --Order Date
  ,ord_tax_loc   INT --Order Tax Location (State)
  ,order_tot     NUMERIC(12,2) --Order Total (all lines)
  ,ord_ship_add  BOOLEAN --Order Ship Address is same as ordering customer
  ,ord_ship_date DATE --Order Ship Date
  ,ord_track_num TEXT --Order Tracking Number
  ,warehouse_id  INT  REFERENCES dsci_504.warehouses(warehouse_id) --Integer Warehouse ID order shipped from
  ,cus_id        INT  NOT NULL REFERENCES dsci_504.customers(cus_id) --Integer Customer ID placing order
);

CREATE TABLE

Total execution time: 00:00:00.025

In [12]:
-- 11) ORDER_ITEMS (detail)
CREATE TABLE dsci_504.order_items (
  ord_id      INT  NOT NULL REFERENCES dsci_504.orders(ord_id) --Integer Order ID
  ,prod_id    INT  NOT NULL REFERENCES dsci_504.products(prod_id) --Integer Product ID
  ,quantity   INT  NOT NULL --Quantity ordered
  ,line_total NUMERIC(12,2) NOT NULL --Total for item in line on order
  ,PRIMARY KEY (ord_id, prod_id)
);

CREATE TABLE

Total execution time: 00:00:00.115

In [13]:
-- 12) RETURNS (header)
CREATE TABLE dsci_504.returns (
  rac_id           SERIAL PRIMARY KEY --Return Authorization ID
  ,ord_id           INT    NOT NULL REFERENCES dsci_504.orders(ord_id) --Integer Order ID
  ,cus_id           INT    NOT NULL REFERENCES dsci_504.customers(cus_id) --Integer Customer ID
  ,warehouse_id     INT    NOT NULL REFERENCES dsci_504.warehouses(warehouse_id) --Integer Warehouse ID
  ,return_date      DATE   NOT NULL --Return Date (date approved)
  ,tot_ret_item_cnt INT --Number of items returned
  ,tot_ret_amnt     NUMERIC(12,2) --Total of all returned items
);

CREATE TABLE

Total execution time: 00:00:00.039

In [14]:
-- 13) RETURN_ITEMS (detail)
CREATE TABLE dsci_504.return_items (
  rac_id        INT     NOT NULL REFERENCES dsci_504.returns(rac_id) --RAC ID
  ,prod_id       INT     REFERENCES dsci_504.products(prod_id) --Integer Product ID
  ,comp_id       INT     REFERENCES dsci_504.components(comp_id) -- Integer Component ID
  ,return_qty    INT --Number of items returned on the line
  ,return_amount NUMERIC(12,2) --Amount of item
  ,return_reason TEXT --Returning reason (binned)
  ,PRIMARY KEY (rac_id, prod_id, comp_id)
);

CREATE TABLE

Total execution time: 00:00:00.055

In [15]:
-- 14) TAXES (lookup)
CREATE TABLE dsci_504.taxes (
  tax_id   SERIAL PRIMARY KEY --Integer ID
  ,tax_desc TEXT --Description of tax location (state)
  ,tax_rate NUMERIC(5,4) --Tax rate in whole percent
);

CREATE TABLE

Total execution time: 00:00:00.012

In [16]:
-- 15) ORDER_TAXES (bridge Orders ↔ Taxes)
CREATE TABLE dsci_504.order_taxes (
  ord_id INT NOT NULL REFERENCES dsci_504.orders(ord_id) --Integer Order ID
  ,tax_id INT NOT NULL REFERENCES dsci_504.taxes(tax_id) --Integer Tax ID
  ,PRIMARY KEY (ord_id, tax_id)
);

CREATE TABLE

Total execution time: 00:00:00.011

**Step 3: Import the data into the OPC tables in PostgreSQL. Use a new code cell for each file.**

In [17]:
--Enter your SQL code here. 
-- 1) Import data into dsci_504.states table
COPY dsci_504.states(state_id, state)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\states.csv'
DELIMITER ','
CSV HEADER;

COPY 31

Total execution time: 00:00:00.035

In [18]:
-- 2) Import data into dsci_504.customers table
COPY dsci_504.customers(cus_id, cus_num, cus_last_name, cus_first_name, cus_add_num, cus_address, 
cus_city, state_id, cus_zip, cus_phone, cus_join_date, cus_app_cd, cus_app_num)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\customers.csv'
DELIMITER ','
CSV HEADER;

COPY 2619

Total execution time: 00:00:00.484

In [19]:
-- 3) Import data into dsci_504.warehouses table
COPY dsci_504.warehouses(warehouse_id, warehouse_name, state_id)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\warehouses.csv'
DELIMITER ','
CSV HEADER;

COPY 3

Total execution time: 00:00:00.005

In [20]:
-- 4) Import data into dsci_504.suppliers table
COPY dsci_504.suppliers(sup_id, sup_name, sup_ctry)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\suppliers.csv'
DELIMITER ','
CSV HEADER;

COPY 72

Total execution time: 00:00:00.009

In [21]:
-- 5) Import data into dsci_504.components table
COPY dsci_504.components(comp_id, comp_name, comp_cost, sup_id, comp_cat)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\components.csv'
DELIMITER ','
CSV HEADER;

COPY 271

Total execution time: 00:00:00.015

In [22]:
-- 6) Import data into dsci_504.builds table
COPY dsci_504.builds(build_id, build_name)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\builds.csv'
DELIMITER ','
CSV HEADER;

COPY 80

Total execution time: 00:00:00.008

In [23]:
-- 7) Import data into dsci_504.build_components table
COPY dsci_504.build_components(build_id, comp_id)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\build_components.csv'
DELIMITER ','
CSV HEADER;

COPY 691

Total execution time: 00:00:00.393

In [28]:
-- 8) Import data into dsci_504.products table
COPY dsci_504.products(prod_id,prod_cat_name,prod_manufacturer,prod_name,prod_description,prod_price,prod_class,country_origin)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\products.csv'
DELIMITER ','
CSV HEADER;

COPY 69

Total execution time: 00:00:00.005

In [29]:
-- 9) Import data into dsci_504.product_builds table
COPY dsci_504.product_builds(prod_id, build_id)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\product_builds.csv'
DELIMITER ','
CSV HEADER;

COPY 69

Total execution time: 00:00:00.020

In [30]:
-- 10) Import data into dsci_504.orders table
COPY dsci_504.orders(ord_id, ord_date, ord_tax_loc, order_tot, ord_ship_add, 
ord_ship_date, ord_track_num, warehouse_id, cus_id)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\orders.csv'
DELIMITER ','
CSV HEADER;

COPY 2599

Total execution time: 00:00:00.222

In [31]:
-- 11) Import data into dsci_504.order_items table
COPY dsci_504.order_items(ord_id, prod_id, quantity, line_total)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\order_items.csv'
DELIMITER ','
CSV HEADER;

COPY 2599

Total execution time: 00:00:00.240

In [32]:
-- 12) Import data into dsci_504.returns table
COPY dsci_504.returns(rac_id, ord_id, cus_id, warehouse_id, return_date, tot_ret_item_cnt, tot_ret_amnt)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\returns.csv'
DELIMITER ','
CSV HEADER;

: date/time field value out of range: "0"

Total execution time: 00:00:00.005

Challenges: Table schema had \`return\_date\` column, but this column is missing in the data file

In [33]:
-- 13) Import data into dsci_504.return_items table
COPY dsci_504.return_items(rac_id, prod_id, comp_id, return_qty, return_amount, return_reason)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\return_items.csv'
DELIMITER ','
CSV HEADER;

: missing data for column "return_reason"

Total execution time: 00:00:00.005

In [34]:
-- 14) Import data into dsci_504.taxes table
COPY dsci_504.taxes(tax_id, tax_desc, tax_rate)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\taxes.csv'
DELIMITER ','
CSV HEADER;

COPY 31

Total execution time: 00:00:00.013

In [35]:
-- 15) Import data into dsci_504.order_taxes table
COPY dsci_504.order_taxes(ord_id, tax_id)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\order_taxes.csv'
DELIMITER ','
CSV HEADER;

: extra data after last expected column

Total execution time: 00:00:00.005

**Step 4: Execute a simple query on each table to verify the data in each table. Do not worry about missing data at this point. Limit your output to 10 lines using the following query syntax:**

```
SELECT * FROM [table] LIMIT 5;

```

In [ ]:
-- 1) Select top 10 records from dsci_504.states table
SELECT * FROM dsci_504.states LIMIT 10;

In [ ]:
-- 2) Select top 10 records from dsci_504.customers table
SELECT * FROM dsci_504.customers LIMIT 10;

**Step 5: Create primary keys within each of the newly created tables to elevate the database to 2NF. Some tables already have unique key columns that simply need to be identified as a Primary Key. (If you did this above when building the tables, simply run the below query for each table).Others may require a new column to do this. Run the below query after each table to validate the contstraints are placed on the tables.**

```

SELECT con.*
    FROM pg_catalog.pg_constraint con
        INNER JOIN pg_catalog.pg_class rel ON rel.oid = con.conrelid
        INNER JOIN pg_catalog.pg_namespace nsp ON nsp.oid = connamespace
        WHERE nsp.nspname = '{dsci_504}'
             AND rel.relname = '{states}';

```

**Step 6: CREATE foreign key associations for each table to elevate the database to 3NF. These should be done within each respective table. If this was completed above, note so and move on to Step 7.**

In [33]:
--Enter your SQL code here

Total execution time: 00:00:00

**Step 7: Execute two queries of your choosing on two different tables in the database.**

In [34]:
--Enter SQL code here

Total execution time: 00:00:00

In [35]:
--Enter SQL code here

Total execution time: 00:00:00